In this notebook we attempt to create the knowledge graph based on the test_data.csv using the information obtained during the EDA

In [1]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\Timo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Import important libraries
import pandas as pd
import rdflib
from rdflib import Graph, Namespace, RDF, RDFS, XSD, OWL, Literal
from rdflib.namespace import FOAF
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, URIRef
from datetime import datetime

In [3]:
test_df = pd.read_csv('test_data.csv')
df = pd.read_csv('clinical_trials.csv')

In [4]:
import numpy as np

# Divide the dataframe into 4 parts
df_parts = np.array_split(df, 4)

# Assign each part to a new dataframe
df_part1 = df_parts[0]
df_part2 = df_parts[1]
df_part3 = df_parts[2]
df_part4 = df_parts[3]


In [5]:
test_df.head()

,NCT Number,Study Title,Study URL,Acronym,Study Status,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,...,Study Design,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents
0,NCT00072579,Sargramostim in Treating Patients With Chronic...,https://clinicaltrials.gov/study/NCT00072579,NaN,COMPLETED,"RATIONALE: Colony-stimulating factors, such as...",NO,Leukemia,BIOLOGICAL: sargramostim,Cytogenetic response (complete and partial),...,Allocation: |Intervention Model: |Masking: NON...,CCCWFU-23102|CDR0000340983|BRLX-02153|NCI-7350,2003-05,2006-04,2007-12,2003-11-05,NaN,2017-01-19,"CCOP - Western Regional, Arizona, Phoenix, Ari...",NaN
1,NCT01402479,An Open-labeled Trial of Ramipril in Patients ...,https://clinicaltrials.gov/study/NCT01402479,NaN,COMPLETED,Physiology of migraine involving renin-angiote...,NO,Migraine With Hypertension,DRUG: Ramipril,"headache frequency, headache days, 12 week",...,Allocation: NON_RANDOMIZED|Intervention Model:...,0408-131-005,2004-10,2005-07,2005-07,2011-07-26,NaN,2011-08-08,"Seoul National University Hospital, Seoul, 110...",NaN
2,NCT05600179,OCTA in Epivascular Glia After Dex Implant,https://clinicaltrials.gov/study/NCT05600179,NaN,COMPLETED,The aim of this prospective study was for the ...,NO,Diabetic Retinopathy,DRUG: Dexamethasone intravitreal implant,Changes in epivascular glia after Dexamethason...,...,Observational Model: |Time Perspective: p,0110/2022,2021-01-01,2022-09-30,2022-09-30,2022-10-31,NaN,2022-12-08,"University of Naples ""Federico II"", Naples, 80...",NaN
3,NCT01511679,Brain-imaging and Adolescent Neuroscience Cons...,https://clinicaltrials.gov/study/NCT01511679,BANC,WITHDRAWN,This is a multi-site study of adolescents 12-2...,NO,Alcohol Abuse,NaN,NaN,...,Observational Model: |Time Perspective: p,BANC RFA-AA-12-006,2012-09,2017-03,2017-09,2012-01-19,NaN,2013-01-03,"McLean Hospital, Belmont, Massachusetts, 02478...",NaN
4,NCT05602779,Leverage Noninvasive Transcutaneous Vagus Nerv...,https://clinicaltrials.gov/study/NCT05602779,NaN,RECRUITING,"Suicidal thoughts, suicide attempts, and suici...",NO,Self Harm|Suicidal Ideation,DEVICE: tVns Program|OTHER: Phone App Program|...,Change from Baseline in Non-Suicidal Self Inju...,...,Allocation: RANDOMIZED|Intervention Model: PAR...,22-08-7372,2023-10-08,2027-04-15,2027-09-30,2022-11-02,NaN,2023-12-06,"University of Notre Dame, South Bend, Indiana,...",NaN


Some changes will have to be made to the dataframe to obtain the final runnable data for the graph
- The columns x conditions, x interventions, study design, x age, x phase, x other IDs, locations, x Study documents have multiple entries per row
- The columns 


To Do locations and study design

# Create a graph and add the triples


## good one


In [6]:
from rdflib import Graph, Namespace, URIRef, Literal, XSD
def GraphDracula(dataframe):
    # Define namespaces
    CT = Namespace("http://example.org/clinicalTrial/")
    XSD = Namespace("http://www.w3.org/2001/XMLSchema#")

    # Create an RDF graph
    g = Graph()

    # Bind namespaces
    g.bind("ct", CT)
    g.bind("xsd", XSD)

    # Iterate through DataFrame rows to create RDF triples
    for index, row in dataframe.iterrows():
        # Define URIRef for clinical trial
        clinical_trial_uri = URIRef(CT + "Trial/" + str(row['NCT Number']))
        g.add((clinical_trial_uri, RDF.type, CT.ClinicalTrial))
        # Add triples for Study Information
        g.add((clinical_trial_uri, CT.studyTitle, Literal(row['Study Title'], datatype=XSD.string)))

        g.add((clinical_trial_uri, CT.studyURL, URIRef(row['Study URL'])))
        
        acronym = row['Acronym']
        if not pd.isna(acronym):
            g.add((clinical_trial_uri, CT.acronym, Literal(row['Acronym'], datatype=XSD.string)))

        g.add((clinical_trial_uri, CT.studyStatus, Literal(row['Study Status'], datatype=XSD.string)))
        
        summary = row['Brief Summary']
        if not pd.isna(summary):
            g.add((clinical_trial_uri, CT.briefSummary, Literal(row['Brief Summary'], datatype=XSD.string)))
        
        # Study results were originally in yes or no format so we convert them to boolean
        Study_results = row['Study Results']
        Study_results_bool = True if Study_results == "Yes" else False
        g.add((clinical_trial_uri, CT.hasStudyResults, Literal(Study_results_bool, datatype=XSD.boolean)))
        
        startDate = row['Start Date']
        if not pd.isna(startDate):
            g.add((clinical_trial_uri, CT.startDate, Literal(row['Start Date'], datatype=XSD.date)))
        
        prinCompDate = row['Primary Completion Date']
        if not pd.isna(prinCompDate):
            g.add((clinical_trial_uri, CT.primaryCompletionDate, Literal(row['Primary Completion Date'], datatype=XSD.date)))
        
        compDate = row['Completion Date']
        if not pd.isna(compDate):
            g.add((clinical_trial_uri, CT.completionDate, Literal(row['Completion Date'], datatype=XSD.date)))
        
        g.add((clinical_trial_uri, CT.firstPosted, Literal(row['First Posted'], datatype=XSD.date)))
    
        resultsfirstposted = row['Results First Posted']
        if not pd.isna(resultsfirstposted):
            g.add((clinical_trial_uri, CT.resultsFirstPosted, Literal(row['Results First Posted'], datatype=XSD.date)))
        
        g.add((clinical_trial_uri, CT.lastUpdatePosted, Literal(row['Last Update Posted'], datatype=XSD.date)))
        # There are multiple documents for a trial sometimes so we split them and add them as separate triples
        documents = row['Study Documents']
        if not pd.isna(documents):
            split_documents = documents.split('|')
            for document in split_documents:
                document_parts = document.split(',')
                # I want to split the document into the filename and the filelink
                if len(document_parts) == 2:
                    filename, filelink = document_parts
                    studyDocs = URIRef(CT + str(row['NCT Number']) + "/StudyDocs")
                    g.add((clinical_trial_uri, CT.hasStudyDocuments, studyDocs))
                    g.add((studyDocs, CT.filename, Literal(filename.strip(), datatype=XSD.string)))
                    g.add((studyDocs, CT.fileLink, URIRef(filelink.strip())))

        
        # Add triples for Participant Information
        participant_info = URIRef(CT + str(row['NCT Number']) + "/ParticipantInfo")
        g.add((participant_info, RDF.type, CT.ParticipantInfo))
        g.add((clinical_trial_uri, CT.hasParticipantInfo, participant_info))
        
        sex = row['Sex']
        if not pd.isna(sex):
            g.add((participant_info, CT.sex, Literal(row['Sex'], datatype=XSD.string)))
    
        # age can have multiple values so we split them and add them as separate triples
        age = row['Age']
        if not pd.isna(age):
            for age_item in age.split(','):
                g.add((participant_info, CT.age, Literal(age_item.strip(), datatype=XSD.string)))
        
        enrollment = row['Enrollment']
        if not pd.isna(enrollment):
            enrollint = int(row['Enrollment'])
            g.add((participant_info, CT.enrollment, Literal(enrollint, datatype=XSD.integer)))

        # Add triples for Medical Details
        medical_details = URIRef(CT + str(row['NCT Number']) + "/MedicalDetails")
        g.add((medical_details, RDF.type, CT.MedicalDetails))
        g.add((clinical_trial_uri, CT.hasMedicalDetails, medical_details))
        
        # Conditions can have multiple values so we split them and add them as separate triples
        condition = row['Conditions']
        if not pd.isna(condition):
            for condition_item in condition.split('|'):
                g.add((medical_details, CT.conditions, Literal(condition_item.strip(), datatype=XSD.string)))
        
        #There different types of interventions so we split them and add them as separate triples
        intervention_details = URIRef(CT + str(row['NCT Number']) + "/MedicalDetails/InterventionDetails")
        interventions = row['Interventions']
        if not pd.isna(interventions):
            split_interventions = interventions.split('|')
            for column in split_interventions:
                g.add((medical_details, CT.interventions, Literal(column, datatype=XSD.string)))
        
        Outcome1 = row['Primary Outcome Measures']
        if not pd.isna(Outcome1):
            g.add((medical_details, CT.primaryOutcomeMeasures, Literal(row['Primary Outcome Measures'], datatype=XSD.string)))
        
        Outcome2 = row['Secondary Outcome Measures']
        if not pd.isna(Outcome2):
            g.add((medical_details, CT.secondaryOutcomeMeasures, Literal(row['Secondary Outcome Measures'], datatype=XSD.string)))
        
        Outcome3 = row['Other Outcome Measures']
        if not pd.isna(Outcome3):
            g.add((medical_details, CT.otherOutcomeMeasures, Literal(row['Other Outcome Measures'], datatype=XSD.string)))
        
        phase = row['Phases']
        #Phase has 2 options so we split them and add them as separate triples
        if not pd.isna(phase):
            for phase_item in phase.split('|'):
                g.add((medical_details, CT.phases, Literal(phase_item.strip(), datatype=XSD.string)))
        
        studyT = row['Study Type']
        if not pd.isna(studyT):
            g.add((medical_details, CT.studyType, Literal(row['Study Type'], datatype=XSD.string)))
        
        #There are 2 types of studies INTERVENTIONAL and OBSERVATIONAL these are treated differently
        study_design = URIRef(CT + str(row['NCT Number']) + "/MedicalDetails/StudyDesign")
        studyD = row['Study Design']
        if not pd.isna(studyD):
            split_design = studyD.split('|')
            if row['Study Type'] == 'INTERVENTIONAL':
                g.add((medical_details, CT.designAll, Literal(split_design[0], datatype=XSD.string)))
                g.add((medical_details, CT.designInt, Literal(split_design[1], datatype=XSD.string)))
                g.add((medical_details, CT.designMask, Literal(split_design[2], datatype=XSD.string)))
                g.add((medical_details, CT.designPurp, Literal(split_design[3], datatype=XSD.string)))   
            if row['Study Type'] == 'OBSERVATIONAL':
                g.add((medical_details, CT.designObs, Literal(split_design[0], datatype=XSD.string)))
                g.add((medical_details, CT.designTime, Literal(split_design[1], datatype=XSD.string)))

        
        otherIDs = row['Other IDs']
        # can have multiple other IDs so we split them and add them as separate triples
        if not pd.isna(otherIDs):
            for id_item in otherIDs.split('|'):
                g.add((medical_details, CT.otherIDs, Literal(id_item.strip(), datatype=XSD.string)))

        # Add triples for Collaboration Details
        collaboration_details = URIRef(CT + str(row['NCT Number']) + "/CollaborationDetails")
        g.add((collaboration_details, RDF.type, CT.CollaborationDetails))
        g.add((clinical_trial_uri, CT. , collaboration_details))
        
        g.add((collaboration_details, CT.sponsor, Literal(row['Sponsor'], datatype=XSD.string)))
        collaborators = row['Collaborators'] 
        if not pd.isna(collaborators):
            for collab_item in collaborators.split('|'):
                g.add((collaboration_details, CT.collaborators, Literal(collab_item, datatype=XSD.string)))
        
        fType = row['Funder Type']
        if not pd.isna(fType):
            g.add((collaboration_details, CT.funderType, Literal(row['Funder Type'], datatype=XSD.string)))

        # Add triples for Location Information
        location_info = URIRef(CT + str(row['NCT Number']) + "/LocationInfo")
        g.add((location_info, RDF.type, CT.LocationInfo))
        g.add((clinical_trial_uri, CT.hasLocationInfo, location_info))
        location = row['Locations']
        if not pd.isna(location):
            splitlocations = location.split('|')
            all_locations = []
            for column in splitlocations:
                g.add((location_info, CT.locations, Literal(column, datatype=XSD.string)))
    return g          
            

    


In [7]:
# test_graph = GraphDracula(test_df)
# test_graph.serialize(destination= 'test_graph.ttl',format='turtle')

Because of a memoryerror we split the file

In [8]:
# CTgraph1 = GraphDracula(df_part1)
# CTgraph2 = GraphDracula(df_part2)
# CTgraph3 = GraphDracula(df_part3)
# CTgraph4 = GraphDracula(df_part4)
# CTgraph = CTgraph1 + CTgraph2 + CTgraph3 + CTgraph4
# CTgraph.serialize(destination= 'CT_graph.ttl',format='turtle')